In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

Data source: https://www.kaggle.com/datasets/matthiaskleine/european-mortality-dataset'

In [2]:
df = pd.read_csv("../data/data_raw/european_yearly_deaths_by_country_and_1_year_age_groups.csv")
df

country  Austria  Austria.1  Austria.2  Austria.3  Austria.4  Austria.5  \
0     age      0.0        1.0        2.0        3.0        4.0        5.0   
1    2013    245.0       21.0       15.0       11.0        7.0        5.0   
2    2014    249.0       15.0       13.0       13.0        6.0        7.0   
3    2015    259.0       16.0       15.0        4.0        9.0        8.0   
4    2016    269.0       24.0       12.0       10.0        4.0       10.0   
5    2017    256.0       20.0       14.0        6.0       18.0        7.0   
6    2018    231.0       16.0       12.0        8.0        7.0        6.0   
7    2019    250.0       25.0       19.0       14.0        7.0        8.0   
8    2020    262.0       17.0       15.0        8.0       10.0        4.0   
9    2021    235.0       15.0       13.0        9.0        8.0       10.0   

   Austria.6  Austria.7  Austria.8  Austria.9  Austria.10  Austria.11  \
0        6.0        7.0        8.0        9.0        10.0        11.0   
1        9.0        5.0        8.0        9.0         6.0        12.0   
2        7.0        9.0        9.0        6.0         7.0        10.0   
3        5.0        3.0        4.0        5.0         5.0        13.0   
4        2.0        9.0        5.0        8.0         6.0         7.0   
5        5.0        5.0        5.0        4.0         6.0         5.0   
6        3.0        5.0        8.0        9.0         3.0         5.0   
7        5.0        6.0        3.0        9.0        10.0         7.0   
8        6.0        2.0        7.0        8.0         5.0        10.0   
9        4.0        5.0        3.0        3.0         3.0         3.0   

   Austria.12  Austria.13  Austria.14  Austria.15  Austria.16  Austria.17  \
0        12.0        13.0        14.0        15.0        16.0        17.0   
1         5.0        13.0        11.0        14.0        23.0        24.0   
2         6.0         8.0        11.0        12.0        26.0        31.0   
3        10.0         9.0         8.0        13.0        37.0        21.0   
4         8.0         6.0         6.0        15.0        20.0        22.0   
5        10.0         9.0        10.0        13.0        21.0        21.0   
6         6.0        10.0         9.0        15.0        19.0        28.0   
7         5.0        11.0         8.0        17.0        21.0        19.0   
8         8.0         9.0         6.0        18.0        17.0        29.0   
9         8.0         5.0        10.0        28.0        15.0        27.0   

   Austria.18  Austria.19  Austria.20  Austria.21  Austria.22  Austria.23  \
0        18.0        19.0        20.0        21.0        22.0        23.0   
1        31.0        32.0        42.0        31.0        48.0        50.0   
2        22.0        54.0        40.0        35.0        36.0        45.0   
3        28.0        27.0        51.0        45.0        38.0        49.0   
4        38.0        43.0        51.0        40.0        43.0        31.0   
5        32.0        38.0        32.0        35.0        32.0        33.0   
6        35.0        23.0        27.0        50.0        41.0        41.0   
7        35.0        24.0        32.0        44.0        41.0        25.0   
8        26.0        32.0        36.0        34.0        25.0        36.0   
9        30.0        31.0        41.0        37.0        34.0        41.0   

   Austria.24  Austria.25  Austria.26  Austria.27  Austria.28  Austria.29  \
0        24.0        25.0        26.0        27.0        28.0        29.0   
1        48.0        39.0        55.0        34.0        46.0        56.0   
2        41.0        44.0        33.0        42.0        47.0        69.0   
3        48.0        40.0        42.0        41.0        51.0        46.0   
4        32.0        43.0        50.0        33.0        58.0        42.0   
5        42.0        45.0        57.0        32.0        66.0        45.0   
6        30.0        40.0        46.0        58.0        39.0        58.0   
7        43.0        53.0        39.0 

In [3]:
df = df.T
df.reset_index(inplace=True)

#Headers
headers = df.iloc[0].values
df.columns = headers

In [4]:
df[['country', 'number']] = df['country'].str.split('.', 1, expand=True)
df


,country,age,2013,2014,2015,2016,2017,2018,2019,2020,2021,number
0,country,age,2013,2014,2015,2016,2017,2018,2019,2020,2021,None
1,Austria,0.0,245.0,249.0,259.0,269.0,256.0,231.0,250.0,262.0,235.0,None
2,Austria,1.0,21.0,15.0,16.0,24.0,20.0,16.0,25.0,17.0,15.0,1
3,Austria,2.0,15.0,13.0,15.0,12.0,14.0,12.0,19.0,15.0,13.0,2
4,Austria,3.0,11.0,13.0,4.0,10.0,6.0,8.0,14.0,8.0,9.0,3
...,...,...,...,...,...,...,...,...,...,...,...,...
2622,Switzerland,96.0,895.0,901.0,961.0,1079.0,1200.0,1147.0,1230.0,1387.0,1327.0,96
2623,Switzerland,97.0,675.0,721.0,734.0,635.0,866.0,952.0,976.0,1112.0,1002.0,97
2624,Switzerland,98.0,528.0,507.0,576.0,509.0,583.0,635.0,693.0,780.0,755.0,98
2625,Switzerland,99.0,422.0,355.0,414.0,350.0,360.0,361.0,504.0,594.0,586.0,99


In [5]:
df.drop(['number', 'age'], axis=1, inplace=True)
df.drop(0, axis=0, inplace=True)

In [6]:
df.columns

Index(['country', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021'],
      dtype='object')

In [7]:
# columna a integer:
def to_float(num):
    """
    Convierte una cadena que representa un número a tipo de dato integer

    Args:
        num (str): Una cadena que representa un número entero.

    Returns:
        int: El número equivalente en formato de tipo integer.
    """
    return float(num)

In [8]:
# columna a integer:
def to_int(num):
    """
    Convierte una cadena que representa un número a tipo de dato integer

    Args:
        num (str): Una cadena que representa un número entero.

    Returns:
        int: El número equivalente en formato de tipo integer.
    """
    return int(num)

In [9]:
columns_to_int =['2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020', '2021']

In [10]:
for column in columns_to_int:
    df[column] = df[column].apply(to_float).astype(float)
    df[column] = df[column].apply(to_int).astype(int)

In [11]:
df = df.groupby('country').sum()
df

,2013,2014,2015,2016,2017,2018,2019,2020,2021
country,,,,,,,,,
Austria,79526,78252,83073,80669,83270,83975,83386,91599,91962
Belgium,109334,104755,110541,108097,109666,110693,108783,126896,112331
Bulgaria,104345,108952,110117,107580,109791,108526,108083,124735,148995
Czechia,109160,105665,111173,107750,111443,112920,112362,129289,139891
Denmark,52470,51340,52555,52824,53261,55232,53958,54645,57152
Estonia,15244,15484,15243,15392,15543,15751,15401,15811,18587
Finland,51472,52186,52492,53923,53722,54527,53949,55488,57659
France,569365,559435,593807,594005,606410,609747,613388,669064,661779
Germany,893825,868356,925200,910902,932272,954874,939520,985572,1023687


In [12]:
df.to_csv('../data/data_countries/countries_deaths_year.csv')

In [13]:
df=df.T
df

country,Austria,Belgium,Bulgaria,Czechia,Denmark,Estonia,Finland,France,Germany,Iceland,Italy,Latvia,Liechtenstein,Lithuania,Luxembourg,Malta,Netherlands,Norway,Poland,Portugal,Serbia,Slovakia,Slovenia,Spain,Sweden,Switzerland
2013,79526,109334,104345,109160,52470,15244,51472,569365,893825,2154,600744,28691,246,41511,3822,3236,141245,41131,387312,106554,100300,52089,19334,388600,90402,64961
2014,78252,104755,108952,105665,51340,15484,52186,559435,868356,2049,598364,28466,268,40252,3841,3270,139223,40369,376467,104843,101247,51346,18886,393734,88976,63938
2015,83073,110541,110117,111173,52555,15243,52492,593807,925200,2178,647571,28478,252,41776,3983,3442,147134,40676,394921,108539,103678,53826,19834,420408,90907,67606
2016,80669,108097,107580,107750,52824,15392,53923,594005,910902,2309,615261,28580,271,41106,3967,3342,148997,40726,388009,110573,100834,52351,19689,408231,90982,64964
2017,83270,109666,109791,111443,53261,15543,53722,606410,932272,2238,649061,28757,249,40142,4263,3571,150214,40774,402852,109758,103722,53914,20509,422037,91972,66971
2018,83975,110693,108526,112920,55232,15751,54527,609747,954874,2254,633133,28820,274,39574,4318,3688,153363,40840,414200,113051,101655,54293,20485,425153,92185,67088
2019,83386,108783,108083,112362,53958,15401,53949,613388,939520,2275,634417,27719,263,38281,4283,3688,151885,40684,409709,111793,101458,53234,20588,416102,88766,67780
2020,91599,126896,124735,129289,54645,15811,55488,669064,985572,2304,740317,28854,319,43547,4609,4084,168678,40611,477355,123358,116850,59089,24016,492447,98124,76195
2021,91962,112331,148995,139891,57152,18587,57659,661779,1023687,2333,701346,34600,271,47746,4489,4163,170972,42002,519517,124802,136622,73461,23261,449149,91958,71192


In [15]:
df.to_csv('../data/data_countries/countries_deaths_country.csv')